In [3]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


Step: Load data

In [2]:
import csv
import pandas as pd
import numpy as np
import time

import os


from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.pipeline.core import Pipeline, ScheduleRecurrence, Schedule
from azureml.pipeline.steps import PythonScriptStep

from azureml.core import Workspace, Dataset

from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import Pipeline, PipelineData, PublishedPipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule

from azureml.core import Workspace, Dataset, Datastore

# Assuming you have already set up your workspace
workspace = Workspace.from_config()
datastore = workspace.get_default_datastore()

UserErrorException: UserErrorException:
	Message: You are currently logged-in to a93eafc8-c064-4a88-bc01-47e93068b42b tenant. You don't have access to mhanauer subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='Azure subscription 1', subscription_id='d3ade6e0-a7be-4323-ace4-02b7b4da2451')]. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You are currently logged-in to a93eafc8-c064-4a88-bc01-47e93068b42b tenant. You don't have access to mhanauer subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = \n [SubscriptionInfo(subscription_name='Azure subscription 1', subscription_id='d3ade6e0-a7be-4323-ace4-02b7b4da2451')]. \n Please refer to aka.ms/aml-notebook-auth for d

In [48]:
workspace

Workspace.create(name='mhanauer', subscription_id='d3ade6e0-a7be-4323-ace4-02b7b4da2451', resource_group='ml_modeling')

In [41]:
environment = Environment.from_conda_specification("nfl_env", "nfl_env.yaml")

In [42]:
# Create a compute target
compute_name = "matthewhanauer991"
# Define the environment and dependencies for your script


try:
    compute_target = ComputeTarget(workspace=workspace, name=compute_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

In [43]:
# Create a ScriptRunConfig for your script
src = ScriptRunConfig(
    source_directory=".", script="production_injury_data.py", environment=environment
)

In [44]:
# Create a PythonScriptStep
step = PythonScriptStep(
    name="nfl_injury_scraper",
    script_name="production_injury_data.py",
    compute_target=compute_target,
    runconfig=src.run_config,
)

In [45]:
# Create a Pipeline using the PythonScriptStep
pipeline = Pipeline(workspace=workspace, steps=[step])

In [46]:
# Publish the pipeline
published_pipeline = pipeline.publish(name="nfl_injury_scraper_pipeline")

Created step nfl_injury_scraper [0c6eb2f7][126af5ec-d89d-4458-b860-5bf7b4bb74e9], (This step will run and generate new outputs)


In [47]:
from datetime import time


recurrence = ScheduleRecurrence(frequency="Day", interval=1, time_of_day="13:05")
schedule = Schedule.create(
    workspace,
    name="nfl_injury_scraper_schedule",
    pipeline_id=published_pipeline.id,
    experiment_name="nfl_injury_scraper_experiment",
    recurrence=recurrence,
)